In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from keras import backend as K

In [36]:
df_ready = pd.read_csv('df_ready.csv')

In [37]:
X = df_ready.drop(['is_canceled'], axis=1).to_numpy()
y = df_ready['is_canceled'].to_numpy()
scaler = preprocessing.StandardScaler()
X_standardized = scaler.fit_transform(X)

print("Mean Before Standardized:", round(X[:,0].mean()))
print("Standard deviation Before Standardized:", X[:,0].std())
print("\nMean After Standardized:", round(X_standardized[:,0].mean()))
print("Standard deviation After Standardized:", X_standardized[:,0].std())

Mean Before Standardized: 104.0
Standard deviation Before Standardized: 106.86264950916215

Mean After Standardized: 0.0
Standard deviation After Standardized: 1.0


In [54]:
#Build Model
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def create_model():
    network = models.Sequential()
    network.add(Dense(units=128, activation="elu", input_shape=(X.shape[1],)))
    network.add(Dense(units=64, activation="elu", kernel_regularizer = l2(0.3), bias_regularizer = l2(0.3)))
    network.add(Dense(units=32, activation="elu", kernel_regularizer = l2(0.3), bias_regularizer = l2(0.3)))
    network.add(Dense(units=1, activation="sigmoid"))
    
    opt = Adam(lr=0.001, decay = 1e-5, beta_1=0.9, beta_2=0.999, amsgrad=False)
    network.compile(loss="binary_crossentropy",
                    optimizer=opt, 
                    metrics=["accuracy", f1_m])
    return network

network = create_model()


In [55]:
#Model Evaluation

'''
cv = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)
network_score = []
for train_idx, test_idx in cv.split(X_standardized,y):
    network.fit(X_standardized[train_idx], y[train_idx], 
                epochs=10,
                verbose=1,
                batch_size=128)
    y_pred = network.predict(X_standardized[test_idx])
    network_score.append(accuracy_score(y[test_idx], y_pred.round()))
    print(network_score[-1])

network_score = np.array(network_score)
print("\nAccuracy mean:", network_score.mean())
print("Accuracy std:",network_score.std())
'''


   

'\ncv = StratifiedKFold(n_splits=5, random_state=None, shuffle=True)\nnetwork_score = []\nfor train_idx, test_idx in cv.split(X_standardized,y):\n    network.fit(X_standardized[train_idx], y[train_idx], \n                epochs=10,\n                verbose=1,\n                batch_size=128)\n    y_pred = network.predict(X_standardized[test_idx])\n    network_score.append(accuracy_score(y[test_idx], y_pred.round()))\n    print(network_score[-1])\n\nnetwork_score = np.array(network_score)\nprint("\nAccuracy mean:", network_score.mean())\nprint("Accuracy std:",network_score.std())\n'

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, test_size = 0.2)
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
network.fit(x = X_train, y = y_train, epochs = 20, batch_size = 128, validation_split = 0.2, shuffle = True)

Train on 76409 samples, validate on 19103 samples
Epoch 1/20
76409/76409 [==============================] - 6s 84us/sample - loss: 4.3054 - accuracy: 0.8151 - f1_m: 0.7227 - val_loss: 0.4535 - val_accuracy: 0.8248 - val_f1_m: 0.7433
Epoch 2/20
76409/76409 [==============================] - 5s 70us/sample - loss: 0.4311 - accuracy: 0.8240 - f1_m: 0.7369 - val_loss: 0.4151 - val_accuracy: 0.8264 - val_f1_m: 0.7428
Epoch 3/20
76409/76409 [==============================] - 6s 82us/sample - loss: 0.4067 - accuracy: 0.8256 - f1_m: 0.7407 - val_loss: 0.3959 - val_accuracy: 0.8310 - val_f1_m: 0.7493
Epoch 4/20
76409/76409 [==============================] - 6s 80us/sample - loss: 0.3934 - accuracy: 0.8289 - f1_m: 0.7451 - val_loss: 0.3864 - val_accuracy: 0.8323 - val_f1_m: 0.7517
Epoch 5/20
76409/76409 [==============================] - 7s 95us/sample - loss: 0.3838 - accuracy: 0.8333 - f1_m: 0.7533 - val_loss: 0.3816 - val_accuracy: 0.8265 - val_f1_m: 0.7254
Epoch 6/20
76409/76409 [===========

In [64]:
y_pred = network.predict(X_test).ravel().round()
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.92      0.88     15000
           1       0.84      0.74      0.78      8878

    accuracy                           0.85     23878
   macro avg       0.85      0.83      0.83     23878
weighted avg       0.85      0.85      0.85     23878

